**Replication of Aguiar & Gopinath (2007): Emerging Market Business Cycles: The cycle is the trend**

Empirical regularities + Matlab Codes

In [ ]:
import pandas as pd
import numpy as np

#We will work with data from Mexico and Canada, 1980 to 2022, quarterly. Check for sources

#Costruct variables of analysis (taking care of indices).

#Detrending, smoothing and statistics.
